# Turning cs-en translation data into cs-cs paraphrase data using huggingface transformers

In [4]:
!nvidia-smi

Mon May 22 08:10:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install transformers datasets sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We load the tranlsation data from google drive

In [6]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [7]:
from transformers import pipeline
from datasets import load_dataset
from tqdm import tqdm

In [8]:
ds = load_dataset('csv', data_files='/content/drive/MyDrive/data/cs_en_filtered.csv')
ds = ds['train']
ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a02c039f625020c9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['English', 'Czech'],
    num_rows: 76879
})

In [9]:
pipe = pipeline('translation', model='Helsinki-NLP/opus-mt-en-cs', device=0)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
def data():
  for row in ds:
    yield row['English']

In [11]:
ds_length = ds.num_rows
ps = []
for res in tqdm(pipe(data(), batch_size=48), total=ds_length):
  ps.append(res[0]['translation_text'])

100%|██████████| 76879/76879 [36:57<00:00, 34.67it/s]


In [12]:
ds = ds.add_column(name='Paraphrase', column=ps)
ds = ds.rename_column('Czech', 'Original')
ds = ds.remove_columns(['English'])
ds = ds.to_csv('/content/drive/MyDrive/data/paraphrases_cs.csv')

Creating CSV from Arrow format:   0%|          | 0/77 [00:00<?, ?ba/s]